<a href="https://colab.research.google.com/github/carsofferrei/04_data_processing/blob/main/spark/challenges/challenge_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CHALLENGE 1
##  Implement INGESTION process
- Set up path in the "lake"
  - !mkdir -p /content/lake/bronze

- Read data from API https://api.carrismetropolitana.pt/
  - Endpoints:
    - vehicles
    - lines
    - municipalities
  - Use StructFields to enforce schema

- Transformations
  - vehicles
    - create "date" extracted from "timestamp" column (format: hh24miss)

- Write data as PARQUET into the BRONZE layer (/content/lake/bronze)
  - Partition "vehicles" by "date" column
  - Paths:
    - vehicles - path: /content/lake/bronze/vehicles
    - lines - path: /content/lake/bronze/lines
    - municipalities - path: /content/lake/bronze/municipalities
  - Make sure there is only 1 single parquet created
  - Use overwrite as write mode

# Setting up PySpark

In [1]:
%pip install pyspark

In [2]:
# Import SparkSession

from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').appName('Carla_Ferreira_API_challenge').config('spark.ui.port', '4050').getOrCreate()
sc = spark.sparkContext

In [3]:
import requests
from pyspark.sql.types import *

def readFromAPI(url: str, schema: StructType = None):
  response = requests.get(url)
  rdd = sc.parallelize(response.json())

  if schema:
    df = spark.read.schema(schema).json(rdd)
  else:
    df = spark.read.json(rdd)
  return df

In [ ]:
#Schemas
vehicle_schema = StructType([StructField('bearing', IntegerType(), True),
                             StructField('block_id', StringType(), True),
                             StructField('current_status', StringType(), True),
                             StructField('id', StringType(), True),
                             StructField('lat', FloatType(), True),
                             StructField('line_id', StringType(), True),
                             StructField('lon', FloatType(), True),
                             StructField('pattern_id', StringType(), True),
                             StructField('route_id', StringType(), True),
                             StructField('schedule_relationship', StringType(), True),
                             StructField('shift_id', StringType(), True),
                             StructField('speed', FloatType(), True),
                             StructField('stop_id', StringType(), True),
                             StructField('timestamp', TimestampType(), True),
                             StructField('trip_id', StringType(), True)
                             ])


lines_schema = StructType([StructField('_corrupt_record', StringType(), True),
                           StructField('color', StringType(), True),
                           StructField('facilities', StringType(), True),
                           StructField('id', StringType(), True),
                           StructField('localities', StringType(), True),
                           StructField('long_name', StringType(), True),
                           StructField('municipalities', StringType(), True),
                           StructField('patterns', StringType(), True),
                           StructField('routes', StringType(), True),
                           StructField('short_name', StringType(), True),
                           StructField('text_color', StringType(), True)
                           ])

municipalities_schema = StructType([StructField('district_id', StringType(), True),
                                    StructField('district_name', StringType(), True),
                                    StructField('id', StringType(), True),
                                    StructField('name', StringType(), True),
                                    StructField('prefix', StringType(), True),
                                    StructField('region_id', StringType(), True),
                                    StructField('region_name', StringType(), True)
                                    ])

In [6]:
#vehicles = readFromAPI("https://api.carrismetropolitana.pt/vehicles")
#lines = readFromAPI("https://api.carrismetropolitana.pt/lines")
municipalities = readFromAPI("https://api.carrismetropolitana.pt/municipalities")

municipalities.show()

+-----------+-------------+----+--------------------+------+---------+----------------+
|district_id|district_name|  id|                name|prefix|region_id|     region_name|
+-----------+-------------+----+--------------------+------+---------+----------------+
|         07|        Évora|0712|        Vendas Novas|    19|    PT187|Alentejo Central|
|         11|       Lisboa|1101|            Alenquer|    20|    PT16B|           Oeste|
|         11|       Lisboa|1102|   Arruda dos Vinhos|    20|    PT16B|           Oeste|
|         11|       Lisboa|1105|             Cascais|    05|    PT170|             AML|
|         11|       Lisboa|1106|              Lisboa|    06|    PT170|             AML|
|         11|       Lisboa|1107|              Loures|    07|    PT170|             AML|
|         11|       Lisboa|1109|               Mafra|    08|    PT170|             AML|
|         11|       Lisboa|1110|              Oeiras|    12|    PT170|             AML|
|         11|       Lisboa|1111|